In [144]:
import pandas as pd
import torch
from collections import Counter
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [145]:
data=pd.read_csv('/kaggle/input/news-clickbait-dataset/train1.csv')

In [146]:
dic={}
for s in data['headline'][:3200]:
  separated_words=s.split()
  separated_words.append('.')
  for w1 ,w2 in zip(separated_words,separated_words[1:]):
    bigram=(w1,w2)
    dic[bigram]=dic.get(bigram,0)+1

In [147]:
words = set()
for key in dic.keys():
    words.update(key)

words=list(words)
words=sorted(words)
end_token=words.pop(words.index('.'))
words.insert(0,'.')
stoi = {word: index for index, word in enumerate(words)}

In [148]:
xs,ys=[],[]
dic={}
for s in data['headline'][:3200]:
  separated_words=s.split()
  separated_words.append('.')
  for w1 ,w2 in zip(separated_words,separated_words[1:]):
    ix1=stoi[w1]
    ix2=stoi[w2]
    xs.append(ix1)
    ys.append(ix2)
xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [149]:
print(xs)
print(ys)

tensor([4779, 2745, 2322,  ..., 5885,  826, 5351])
tensor([2745, 2322,  862,  ...,  826, 5351,    0])


In [150]:
W = torch.randn((len(words),len(words)),requires_grad=True)

In [151]:
batch=[]
batch_size=32
batch_ans=[]
number_of_batches=xs.numel()/batch_size
for i in range (int(number_of_batches)):
    batch.append(xs[batch_size*i:batch_size*(i+1)][:])
    batch_ans.append(ys[batch_size*i:batch_size*(i+1)][:])
    #if i==int(number_of_batches)-1:
                #  epoch.append(xs[batch_size*(i+1):xs.shape[0]])
                #  epoch_ans.append(ys[batch_size*(i+1):ys.shape[0]])

In [152]:
for k in range (15):
    print(k)
    batch_enc=[]
    for i in range(len(batch)):
        W.grad=None
        #Forward Propagation
        batch_enc.append(F.one_hot(batch[i],num_classes=len(words)).float().requires_grad_(True))
    
    
        logits=(batch_enc[i] @ W)
        counts=logits.exp()
        prob=counts/counts.sum(1,keepdims=True)
        loss=-prob[torch.arange(32),batch_ans[i]].log().mean()+0.01*(W**2).mean()
     #Backward Propagation
        loss.backward() 
        print(loss.item())
    
    #Updation of weights
        W.data += -1 * W.grad
print(loss.item())   
    
    
    

0
9.484132766723633
8.930411338806152
9.572908401489258
9.192882537841797
9.22664737701416
9.137147903442383
9.342096328735352
9.283133506774902
9.106619834899902
9.426440238952637
9.420619010925293
9.054073333740234
8.866198539733887
9.167740821838379
8.948898315429688
8.982151985168457
9.213547706604004
9.201173782348633
9.263378143310547
8.919121742248535
9.447978019714355
9.155978202819824
9.108508110046387
9.360454559326172
9.166956901550293
9.497591972351074
9.230690956115723
9.055648803710938
9.301427841186523
8.914115905761719
9.345726013183594
9.491921424865723
9.322583198547363
9.156733512878418
9.145215034484863
9.2252836227417
9.343819618225098
9.181387901306152
9.077832221984863
8.840118408203125
9.24770736694336
9.787903785705566
9.436639785766602
9.199463844299316
9.27059555053711
9.11611557006836
9.423368453979492
9.32852554321289
9.25073528289795
9.158028602600098
8.770575523376465
9.454331398010254
9.325468063354492
9.11230754852295
9.432765007019043
9.369460105895996

In [195]:
for i in range(10):
  
  out = []
  ix = 0
  for k in range(5):
    xenc = F.one_hot(torch.tensor([ix]), num_classes=len(words)).float()
    logits = xenc @ W 
    counts = logits.exp() 
    p = counts / counts.sum(1, keepdims=True) 
    
    ix = torch.multinomial(p, num_samples=1, replacement=True).item()
    out.append(words[ix])
    
    out.append(' ')
   
  print(''.join(out))

Aerial Francisco Sandcastle Fictions to 
Gasping Uncontrollably Toast "Nude" Tarot 
Guys, Borderline Ig Knope AF 
Paid Zodiac Sign . Nae 
Winners Grande LOL "Grease" Dylan 
Bodily Drop Decorations Rica Beauty 
"Con Ron, Kindness You?" Trolls 
Swanson & Campus Auditioned Home 
Figure Stepping Existence Crap Safe 
Thirteenth Toby Incredibly Harry, Updated 


In [192]:
torch.save(W, 'model_weights.pth')